## Convert bird observations registered in artsobeservasjoner.no into a ebird.org record format

In [89]:
# !pip install pandas
# !pip install pyproj
# !pip install openpyxl

In [90]:
import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Po
import pyproj
from pyproj import CRS, Proj
from pyproj.transformer import Transformer
from pathlib import Path
# import geodatasets

ao_source = "ExcelExport_8077412_Page_1.xlsx"
ebird_template = "ebird_record_format_template.csv"

In [91]:
ao_df = pd.read_excel(ao_source, skiprows=2)
print(list(ao_df.columns))
ao_df[['Superlokalitet', 'Lokalitetsnavn','Østkoordinat', 'Nordkoordinat', 'Nøyaktighet', 'Originale koordinater',]].head(10)
for idx, value in ao_df.loc[1].iteritems():
    print(idx, value)
# print(type(ao_df.loc[0]["Startdato"]))
ao_df.head(4)

['Id', 'Taksonsorteringsrekkefølge', 'Valideringsstatus', 'Rødlistekategori', 'Artsnavn', 'Vitenskapelig navn', 'Autor', 'Skjermet funn', 'Antall', 'Enhet', 'Alder', 'Kjønn', 'Aktivitet', 'Metode', 'Superlokalitet', 'Lokalitetsnavn', 'Østkoordinat', 'Nordkoordinat', 'Nøyaktighet', 'Originale koordinater', 'Fylke', 'Kommune', 'Fylke.1', 'IKKE I NORGE', 'Ekstern id', 'Startdato', 'Stattidspunkt', 'Sluttdato', 'Sluttidspunkt', 'Kommentar', 'Ikke gjenfunnet', 'Usikker artsbestemmelse', 'Uspontan', 'Natursystem', 'Natursystem beskrivelse', 'Livsmedium', 'Vitenskapelig livsmediumnavn', 'Art som livsmedium, beskrivelse', 'Livsmedium.1', 'Livsmediumbeskrivelse', 'Min. dybde', 'Maks. dybde', 'Høyde min', 'Høyde maks', 'Offentlig samling', 'Privat samling', 'Samlingsnummer', 'Samlingsbeskrivelse', 'Artsbestemt av', 'Bestemmelsesdato', 'Bekrefter', 'Bekreftelsesdato', 'Redigeringsansvarlig', 'Rapportør', 'Observatører', 'Prosjekt']
Id 28521511
Taksonsorteringsrekkefølge 70410787
Valideringsstatus

,Id,Taksonsorteringsrekkefølge,Valideringsstatus,Rødlistekategori,Artsnavn,Vitenskapelig navn,Autor,Skjermet funn,Antall,Enhet,...,Samlingsnummer,Samlingsbeskrivelse,Artsbestemt av,Bestemmelsesdato,Bekrefter,Bekreftelsesdato,Redigeringsansvarlig,Rapportør,Observatører,Prosjekt
0,28521548,70413656,Ikke validert (funnet er ikke kvalitetssikret),NaN,fossekall,Cinclus cinclus,"(Linnaeus, 1758)",Nei,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Egil Rønningstad,Egil Rønningstad,"Monica Rønningstad, Egil Rønningstad",NaN
1,28521511,70410787,Ikke validert (funnet er ikke kvalitetssikret),NaN,blåstrupe,Luscinia svecica,"(Linnaeus, 1758)",Nei,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Egil Rønningstad,Egil Rønningstad,"Monica Rønningstad, Egil Rønningstad",NaN
2,28467966,70403698,Ikke validert (funnet er ikke kvalitetssikret),NaN,løvsanger,Phylloscopus trochilus,"(Linnaeus, 1758)",Nei,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Monica Rønningstad,Monica Rønningstad,"Egil Rønningstad, Monica Rønningstad",NaN
3,28467965,70402168,Ikke validert (funnet er ikke kvalitetssikret),NaN,låvesvale,Hirundo rustica,"Linnaeus, 1758",Nei,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Monica Rønningstad,Monica Rønningstad,"Egil Rønningstad, Monica Rønningstad",NaN


In [92]:
ebird_headers = list(pd.read_csv(ebird_template, sep=";").columns)
# Add these headers to the ao list before we populate them
for ebird_header in ebird_headers:
    ao_df["ebird-"+ebird_header]= ""
ebird_headers

['Common Name',
 'Genus',
 'Species',
 'Number',
 'Species Comments',
 'Location Name',
 'Latitude',
 'Longitude',
 'Date',
 'Start Time',
 'State/Province',
 'Country Code',
 'Protocol',
 'Number of Observers',
 'Duration',
 'All observations reported?',
 'Effort Distance Miles',
 'Effort area acres',
 'Submission Comments']

In [93]:
# Tried tp use the Originale koordinater, but that is empty for personal locations
def digitsonly(text:str):
    return int("".join([c for c in text if c.isdigit()]))
def latlong_from_orig(orig_text:str):
    splits = orig_text.split(" ")
    easting = digitsonly(splits[0])
    northing = digitsonly(splits[1])
    zone = digitsonly(splits[3])
    precision = digitsonly(splits[4])
    p = pyproj.Proj(proj='utm', zone=zone, datum='WGS84')
    lon, lat = p(easting, northing, inverse=True)
    return lat, lon

latlong_from_orig("Ø16359, N6954796 Sone 33 (±200m) UTM(WGS 84)")

(62.407573133496484, 5.616887295628967)

In [94]:
ao_df.head(5)

,Id,Taksonsorteringsrekkefølge,Valideringsstatus,Rødlistekategori,Artsnavn,Vitenskapelig navn,Autor,Skjermet funn,Antall,Enhet,...,ebird-Start Time,ebird-State/Province,ebird-Country Code,ebird-Protocol,ebird-Number of Observers,ebird-Duration,ebird-All observations reported?,ebird-Effort Distance Miles,ebird-Effort area acres,ebird-Submission Comments
0,28521548,70413656,Ikke validert (funnet er ikke kvalitetssikret),NaN,fossekall,Cinclus cinclus,"(Linnaeus, 1758)",Nei,1,NaN,...,,,,,,,,,,
1,28521511,70410787,Ikke validert (funnet er ikke kvalitetssikret),NaN,blåstrupe,Luscinia svecica,"(Linnaeus, 1758)",Nei,1,NaN,...,,,,,,,,,,
2,28467966,70403698,Ikke validert (funnet er ikke kvalitetssikret),NaN,løvsanger,Phylloscopus trochilus,"(Linnaeus, 1758)",Nei,1,NaN,...,,,,,,,,,,
3,28467965,70402168,Ikke validert (funnet er ikke kvalitetssikret),NaN,låvesvale,Hirundo rustica,"Linnaeus, 1758",Nei,2,NaN,...,,,,,,,,,,
4,28467963,70418596,Ikke validert (funnet er ikke kvalitetssikret),NaN,tornirisk,Linaria cannabina,"(Linnaeus, 1758)",Nei,2,NaN,...,,,,,,,,,,


In [102]:

c = 'Genus' #,  'Species',
ao_df["ebird-"+c] = ao_df["Vitenskapelig navn"].str.split(" ").str.get(0)
c = 'Species' #,  'Species',
ao_df["ebird-"+c] = ao_df["Vitenskapelig navn"].str.split(" ").str.get(-1)

c = 'Number'
ao_df["ebird-"+c] = ao_df["Antall"]

c = 'Species Comments'
ao_df["ebird-"+c] = ao_df["Aktivitet"]

c =  'Date' # Dates must adhere to the following format: month/day/year (e.g., 12/27/2007).
ao_df["ebird-"+c] = ao_df["Startdato"].dt.strftime('%m/%d/%Y')

c = 'Location Name' # Lokalitetsnavn
ao_df["ebird-"+c] = ao_df["Lokalitetsnavn"]

c = 'Latitude'
ao_df["ebird-"+c] = ao_df["Nordkoordinat"]

c = 'Longitude'
ao_df["ebird-"+c] = ao_df["Østkoordinat"]

c = 'Country Code' # IKKE I NORGE , code "NO"
ao_df.loc[ao_df["IKKE I NORGE"].isnull(),"ebird-"+c] = "NO"

c = 'Protocol'
ao_df["ebird-"+c] = "Historical" #  or Incidental or Stationary https://support.ebird.org/en/support/solutions/articles/48000950859#anchorQuickProtocols

c = 'Number of Observers'
ao_df["ebird-"+c] = ao_df["Observatører"].str.split(",").str.len()

c =  'All observations reported?'
ao_df["ebird-"+c] = "N"

c = 'Submission Comments'
ao_df["ebird-"+c] = "ao_id:"+ ao_df["Id"].apply(str)

In [96]:
len(ao_df), len(ao_df[ao_df["IKKE I NORGE"].isnull()])
# ao_df["Observatører"].str.split(",").str.len()

(30, 30)

In [103]:
ao_df[[c for c in ao_df.columns if c.startswith("ebird")]].head(4)

,ebird-Common Name,ebird-Genus,ebird-Species,ebird-Number,ebird-Species Comments,ebird-Location Name,ebird-Latitude,ebird-Longitude,ebird-Date,ebird-Start Time,ebird-State/Province,ebird-Country Code,ebird-Protocol,ebird-Number of Observers,ebird-Duration,ebird-All observations reported?,ebird-Effort Distance Miles,ebird-Effort area acres,ebird-Submission Comments
0,Cinclus cinclus,Cinclus,cinclus,1,,Spranget,61.839439,9.732312,07/12/2014,,,NO,Historical,2,,N,,,ao_id:28521548
1,Luscinia svecica,Luscinia,svecica,1,,Krokåtbekkbue,61.854240,9.763304,07/13/2014,,,NO,Historical,2,,N,,,ao_id:28521511
2,Phylloscopus trochilus,Phylloscopus,trochilus,1,Sang/spill i hekketid og passende hekkebiotop,Rønningstad,60.873799,11.287466,05/05/2006,,,NO,Historical,2,,N,,,ao_id:28467966
3,Hirundo rustica,Hirundo,rustica,2,,Rønningstad,60.873799,11.287466,05/05/2006,,,NO,Historical,2,,N,,,ao_id:28467965


In [98]:
list(ao_df["Id"])
ebird_path = Path(ao_source).stem+"-ebird.csv"
ebird_df = ao_df[[c for c in ao_df.columns if c.startswith("ebird")]].copy()
convert_back = {"ebird-"+c:c for c in ebird_headers}
ebird_df = ebird_df.rename(convert_back, axis=1)
ebird_df.to_csv(ebird_path, index=False, header=None)
